##Mount Google Drive




In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

## <img src="https://raw.githubusercontent.com/rclone/rclone/master/graphics/logo/svg/logo_symbol_color.svg" height="45" align="left"/><font size=6>rclone</font>

(collapsible / expandable)

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install rclone</font>
build_version = "stable" #@param ["stable", "beta" , "v1.61.1"]

#@markdown ---
automatically_clear_cell_output = True  # @param{type: "boolean"}
# ================================================================ #

import os
import IPython
from IPython.display import clear_output


if build_version == "stable":
	!curl https://rclone.org/install.sh | sudo bash
if build_version == "stable":
	!curl https://rclone.org/install.sh | sudo bash -s beta
if build_version == "v1.61.1":
	!curl -O https://downloads.rclone.org/v1.61.1/rclone-v1.61.1-linux-amd64.zip
	!unzip rclone-v1.61.1-linux-amd64.zip
	!sudo cp rclone-v1.61.1-linux-amd64/rclone /usr/bin/
	!sudo chown root:root /usr/bin/rclone
	!sudo chmod 755 /usr/bin/rclone
	!sudo mkdir -p /usr/local/share/man/man1
	!sudo rclone-v1.61.1-linux-amd64/cp rclone.1 /usr/local/share/man/man1/
	!sudo mandb

!sudo apt-get -y install fuse3

try:
	os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
	pass


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass

### <font size=6>rclone 2</font>

(collapsible / expandable)

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] rclone</font>
Mode = "Copy" #@param ["Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash"]
Source = "" #@param {type:"string"}
Destination = "" #@param {type:"string"}

#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Extra_Arguments = "--local-no-check-updated" #@param {type:"string"}
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Checkers = 10 #@param {type:"slider", min:1, max:40, step:1}
Transfers = 10 #@param {type:"slider", min:1, max:20, step:1}
Dry_Run = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Google_Drive_optimization = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False #@param {type:"boolean"}
Simple_Ouput = True #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]

#@markdown <br><center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <br><center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

##### Importing the needed modules
import os
import IPython
from IPython.display import HTML, clear_output


##### Variable Declaration
# Optimized for Google Colaboratory
os.environ["bufferC"] = "--buffer-size 96M"

if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
else:
    os.environ["compareC"] = ""

os.environ["sourceC"] = Source
os.environ["destinationC"] = Destination
os.environ["transfersC"] = "--transfers "+str(Transfers)
os.environ["checkersC"] = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
else:
    os.environ["skipnewC"] = ""
  
if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
else:
    os.environ["skipexistC"] = ""
  
if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
else:
    os.environ["nocrossfilesystemC"] = ""
  
if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
else:
    os.environ["noupdatemodtimeC"] = ""

if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
else:
    os.environ["filesoptimizeC"] = ""
  
if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
    os.environ["driveoptimizeC"] = ""
  
if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
else:
    os.environ["dryrunC"] = ""
  
if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
    if Simple_Ouput == True:
        os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
        os.environ["statsC"] = "-v --stats=5s"
  
if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
else:
    os.environ["loglevelC"] = ""

os.environ["extraC"] = Extra_Arguments

if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"
  
if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
else:
    os.environ["trackrenamesC"] = ""
  
if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"
  
if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"


##### rclone Execution
if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enabled, rclone will no longer display any output on the terminal.<br>Please wait until the cell stop by itself.</h2></center><br>"))

if Mode == "Copy":
    !rclone --config=/root/.config/rclone/rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Move":
    !rclone --config=/root/.config/rclone/rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Sync":
    !rclone --config=/root/.config/rclone/rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Checker":
    !rclone --config=/root/.config/rclone/rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Deduplicate":
    !rclone --config=/root/.config/rclone/rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.config/rclone/rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == "Empty Trash":
    !rclone --config=/root/.config/rclone/rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC


##### Log Output
if Output_Log_File != "OFF" and Mode != "Config":

    ##### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
        f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))

    ##### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/

    ##### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
        try:
            files.download("/root/rclone_log.zip")
            !rm -f /root/rclone_log.zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
        except:
            !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br><img src=\"https://minormole.github.io/RcloneLab/res/rclonelab/01.png\"><br></center>"))
    else:
        clear_output()
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))
    

### Operation has been successfully completed.
if Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))


##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass

### <font size=6>rclone Configuration</font>

(collapsible / expandable)

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] rclone CLI Configuration</font>
# @markdown <p>Run this cell to create and/or edit an rclone configuration.</p>
# @markdown > After you have created a configuration, download the configuration file.</br>In the next time you want to mount an rclone drive, simply import the configuration file.
USE_FREE_TOKEN = True
TOKEN = ""  #
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]

# @markdown ---
automatically_clear_cell_output = True  # @param{type: "boolean"}

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"
# ================================================================ #

import os, urllib.request, IPython
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    prepareSession,
    rcloneConfigurationPath,
    runSh,
    PortForward_wrapper
)

import codecs, contextlib, locale, os, pty, select, signal, subprocess, sys, termios, time
from IPython.utils import text
import six
from google.colab import _ipython
from google.colab import _message
from google.colab.output import _tags

# Linux read(2) limits to 0x7ffff000 so stay under that for clarity.
_PTY_READ_MAX_BYTES_FOR_TEST = 2**20  # 1MB

_ENCODING = 'UTF-8'

class ShellResult(object):
  """Result of an invocation of the shell magic.

  Note: This is intended to mimic subprocess.CompletedProcess, but has slightly
  different characteristics, including:
    * CompletedProcess has separate stdout/stderr properties. A ShellResult
      has a single property containing the merged stdout/stderr stream,
      providing compatibility with the existing "!" shell magic (which this is
      intended to provide an alternative to).
    * A custom __repr__ method that returns output. When the magic is invoked as
      the only statement in the cell, Python prints the string representation by
      default. The existing "!" shell magic also returns output.
  """

  def __init__(self, args, returncode, command_output):
    self.args = args
    self.returncode = returncode
    self.output = command_output

  def check_returncode(self):
    if self.returncode:
      raise subprocess.CalledProcessError(
          returncode=self.returncode, cmd=self.args, output=self.output)

  def _repr_pretty_(self, p, cycle):  # pylint:disable=unused-argument
    # Note: When invoking the magic and not assigning the result
    # (e.g. %shell echo "foo"), Python's default semantics will be used and
    # print the string representation of the object. By default, this will
    # display the __repr__ of ShellResult. Suppress this representation since
    # the output of the command has already been displayed to the output window.
    if cycle:
      raise NotImplementedError


def _configure_term_settings(pty_fd):
  term_settings = termios.tcgetattr(pty_fd)
  # ONLCR transforms NL to CR-NL, which is undesirable. Ensure this is disabled.
  # http://man7.org/linux/man-pages/man3/termios.3.html
  term_settings[1] &= ~termios.ONLCR

  # ECHOCTL echoes control characters, which is undesirable.
  term_settings[3] &= ~termios.ECHOCTL

  termios.tcsetattr(pty_fd, termios.TCSANOW, term_settings)


def _run_command(cmd, clear_streamed_output):
  """Calls the shell command, forwarding input received on the stdin_socket."""
  locale_encoding = locale.getpreferredencoding()
  if locale_encoding != _ENCODING:
    raise NotImplementedError(
        'A UTF-8 locale is required. Got {}'.format(locale_encoding))

  parent_pty, child_pty = pty.openpty()
  _configure_term_settings(child_pty)

  epoll = select.epoll()
  epoll.register(
      parent_pty,
      (select.EPOLLIN | select.EPOLLOUT | select.EPOLLHUP | select.EPOLLERR))

  try:
    temporary_clearer = _tags.temporary if clear_streamed_output else _no_op

    with temporary_clearer(), _display_stdin_widget(
        delay_millis=500) as update_stdin_widget:
      # TODO(b/115531839): Ensure that subprocesses are terminated upon
      # interrupt.
      p = subprocess.Popen(
          cmd,
          shell=True,
          executable='/bin/bash',
          stdout=child_pty,
          stdin=child_pty,
          stderr=child_pty,
          close_fds=True)
      # The child PTY is only needed by the spawned process.
      os.close(child_pty)

      return _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget)
  finally:
    epoll.close()
    os.close(parent_pty)


class _MonitorProcessState(object):

  def __init__(self):
    self.process_output = six.StringIO()
    self.is_pty_still_connected = True


def _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget):
  """Monitors the given subprocess until it terminates."""
  state = _MonitorProcessState()

  # A single UTF-8 character can span multiple bytes. os.read returns bytes and
  # could return a partial byte sequence for a UTF-8 character. Using an
  # incremental decoder is incrementally fed input bytes and emits UTF-8
  # characters.
  decoder = codecs.getincrementaldecoder(_ENCODING)()

  num_interrupts = 0
  echo_status = None
  while True:
    try:
      result = _poll_process(parent_pty, epoll, p, cmd, decoder, state)
      if result is not None:
        return result
      term_settings = termios.tcgetattr(parent_pty)
      new_echo_status = bool(term_settings[3] & termios.ECHO)
      if echo_status != new_echo_status:
        update_stdin_widget(new_echo_status)
        echo_status = new_echo_status
    except KeyboardInterrupt:
      try:
        num_interrupts += 1
        if num_interrupts == 1:
          p.send_signal(signal.SIGINT)
        elif num_interrupts == 2:
          # Process isn't responding to SIGINT and user requested another
          # interrupt. Attempt to send SIGTERM followed by a SIGKILL if the
          # process doesn't respond.
          p.send_signal(signal.SIGTERM)
          time.sleep(0.5)
          if p.poll() is None:
            p.send_signal(signal.SIGKILL)
      except KeyboardInterrupt:
        # Any interrupts that occur during shutdown should not propagate.
        pass

      if num_interrupts > 2:
        # In practice, this shouldn't be possible since
        # SIGKILL is quite effective.
        raise


def _poll_process(parent_pty, epoll, p, cmd, decoder, state):
  """Polls the process and captures / forwards input and output."""

  terminated = p.poll() is not None
  if terminated:
    termios.tcdrain(parent_pty)
    # We're no longer interested in write events and only want to consume any
    # remaining output from the terminated process. Continuing to watch write
    # events may cause early termination of the loop if no output was
    # available but the pty was ready for writing.
    epoll.modify(parent_pty,
                 (select.EPOLLIN | select.EPOLLHUP | select.EPOLLERR))

  output_available = False

  events = epoll.poll()
  input_events = []
  for _, event in events:
    if event & select.EPOLLIN:
      output_available = True
      raw_contents = os.read(parent_pty, _PTY_READ_MAX_BYTES_FOR_TEST)
      import re
      decoded_contents = re.sub(r"http:\/\/127.0.0.1:53682", Server["url"], 
                                decoder.decode(raw_contents))
      sys.stdout.write(decoded_contents)
      state.process_output.write(decoded_contents)

    if event & select.EPOLLOUT:
      # Queue polling for inputs behind processing output events.
      input_events.append(event)

    # PTY was disconnected or encountered a connection error. In either case,
    # no new output should be made available.
    if (event & select.EPOLLHUP) or (event & select.EPOLLERR):
      state.is_pty_still_connected = False

  for event in input_events:
    # Check to see if there is any input on the stdin socket.
    # pylint: disable=protected-access
    input_line = _message._read_stdin_message()
    # pylint: enable=protected-access
    if input_line is not None:
      # If a very large input or sequence of inputs is available, it's
      # possible that the PTY buffer could be filled and this write call
      # would block. To work around this, non-blocking writes and keeping
      # a list of to-be-written inputs could be used. Empirically, the
      # buffer limit is ~12K, which shouldn't be a problem in most
      # scenarios. As such, optimizing for simplicity.
      input_bytes = bytes(input_line.encode(_ENCODING))
      os.write(parent_pty, input_bytes)

  # Once the process is terminated, there still may be output to be read from
  # the PTY. Wait until the PTY has been disconnected and no more data is
  # available for read. Simply waiting for disconnect may be insufficient if
  # there is more data made available on the PTY than we consume in a single
  # read call.
  if terminated and not state.is_pty_still_connected and not output_available:
    sys.stdout.flush()
    command_output = state.process_output.getvalue()
    return ShellResult(cmd, p.returncode, command_output)

  if not output_available:
    # The PTY is almost continuously available for reading input to provide
    # to the underlying subprocess. This means that the polling loop could
    # effectively become a tight loop and use a large amount of CPU. Add a
    # slight delay to give resources back to the system while monitoring the
    # process.
    # Skip this delay if we read output in the previous loop so that a partial
    # read doesn't unnecessarily sleep before reading more output.
    # TODO(b/115527726): Rather than sleep, poll for incoming messages from
    # the frontend in the same poll as for the output.
    time.sleep(0.1)


@contextlib.contextmanager
def _display_stdin_widget(delay_millis=0):
  """Context manager that displays a stdin UI widget and hides it upon exit.

  Args:
    delay_millis: Duration (in milliseconds) to delay showing the widget within
      the UI.

  Yields:
    A callback that can be invoked with a single argument indicating whether
    echo is enabled.
  """
  shell = _ipython.get_ipython()
  display_args = ['cell_display_stdin', {'delayMillis': delay_millis}]
  _message.blocking_request(*display_args, parent=shell.parent_header)

  def echo_updater(new_echo_status):
    # Note: Updating the echo status uses colab_request / colab_reply on the
    # stdin socket. Input provided by the user also sends messages on this
    # socket. If user input is provided while the blocking_request call is still
    # waiting for a colab_reply, the input will be dropped per
    # https://github.com/googlecolab/colabtools/blob/56e4dbec7c4fa09fad51b60feb5c786c69d688c6/google/colab/_message.py#L100.
    update_args = ['cell_update_stdin', {'echo': new_echo_status}]
    _message.blocking_request(*update_args, parent=shell.parent_header)

  yield echo_updater

  hide_args = ['cell_remove_stdin', {}]
  _message.blocking_request(*hide_args, parent=shell.parent_header)


@contextlib.contextmanager
def _no_op():
  yield

prepareSession()

Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfiguration', 53682, 'http']], 'REGION.lower', [f"{HOME}/.ngrok2/rcloneConfiguration.yml", 4074]).start('rcloneConfiguration', displayB=False, v=False)

printData = """
Before finishing the configuration, you will be redirected to an address.
Replace the address http://127.0.0.0:53682 with {}""".format(Server['url'])
print(printData)
display(HTML('</br>(Click <a href="https://raw.githubusercontent.com/tofuliang/Google-Colab-CloudTorrent/master/src/rclone_config_create.gif" target="_blank">here</a> to see how to do it)</br></br>'))
print(f"{Server['url']}", end="\n\n")
_run_command(f"rclone config --config {rcloneConfigurationPath}/rclone.conf", False)


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount/Unmount rclone Drive (Optional)</font>
# @markdown Mount a remote drive as a local drive on a mountpoint.
# @markdown ---
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #

import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Upload Configuration File </font>
# @markdown <p>If you already have an rclone configuration file, you can upload it by running this cell.</p>

# @markdown ---
MODE = "RCONFIG" # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "mnc" # @param {type:"string"}
QUERY_PATTERN = "" # @param {type:"string"}
# @markdown > <p><font size=2px>For those who are unable to upload local file: <a href="https://stackoverflow.com/a/58661947">StackOverflow</a></font></p>
# ================================================================ #

from os import path as _p
from IPython.display import HTML, clear_output

if not _p.exists("/root/.ipython/mixlab.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(_spl(shellCmd))  # nosec

import importlib, mixlab
from google.colab import files  # pylint: disable=import-error #nosec
from mixlab import checkAvailable, runSh, rcloneConfigurationPath, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {rcloneConfigurationPath}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/mixlab.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{rcloneConfigurationPath}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Upload rclone.conf from your local machine.</h2><br>"))
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nOnly upload one configuration file!")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://shirooo39.github.io/MiXLab/resources/configurations/rclone/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(mixlab)
                    !rm /content/upload.txt
                    clear_output()
                    print("rclone.conf has been uploaded to Colab!")
              return
      else:
          print("\nNo file is chosen!")
          return
    except:
      return print("\nFailed to upload!")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()

## Mega

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← MEGA Login</font>
# @markdown <p>Please log in to MEGA first (only needed to use the Uploader).</p>
# ================================================================ #

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

if not os.path.exists("/root/.ipython/mixlab.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(split(shellCmd))
from mixlab import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# Enter MEGA credential
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please enter your MEGA credential.")

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] MEGA Downloader</font>
URL = "" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
# @markdown > URL: is the MEGA link you want to download (ex: mega.nz/file/file_link#decryption_key)</br>OUTPUT_PATH: is where to store the downloaded file(s) (ex: /content/drive/My Drive/)
# ================================================================ #

import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
)

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
transfare()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] MEGA Uploader</font>
# Simple_torrent = False  # @param{type: "boolean"}
# Peerflix = False  # @param{type: "boolean"}
PATH_TO_FILE = "" # @param {type:"string"}
# @markdown > PATH_TO_FILE is the location of the file you want to upload located at. (ex: /content/downloads/file-to-upload.zip)
# ================================================================ #

import time
import subprocess
import contextlib
from IPython.display import clear_output

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    cmd = ""
    if Simple_torrent:
        cmd = ['mega-put', 'downloads', '/colab']
    elif Peerflix:
        cmd = ['mega-put', 'peerflix', '/colab']
    else:
        cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Please log into your MEGA account first!")

## ZippyShare Downloader

In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← Install the Module and Dependencies</h3>
# ================================================================ #

import os
from IPython.display import clear_output

!git clone https://github.com/mcrapet/plowshare.git /content/plowshare
clear_output()
%cd plowshare
clear_output()
!make install
clear_output()
!plowmod --install
clear_output()
! apt-get install nodejs

path1 = '/content/downloads'
path2 = '/content/downloads/zippyshare'

if os.path.exists(path1) == False:
  os.makedirs(path1)
  if os.path.exists(path2) == False:
    os.makedirs(path2)
  elif os.path.exists(path2) == True:
    None
elif os.path.exists(path1) == True:
  if os.path.exists(path2) == False:
    os.makedirs(path2)
  elif os.path.exists(path2) == True:
    None

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← [Start] zippyshare Downloader</h3></br>
mode = 'single' #@param ["single", "batch"]
# @markdown ---
direct_url = "" #@param {type:"string"}
store_path = "" #@param {type:"string"}
# @markdown > This downloader isn't working as it can't read from zippyshare's weird url (www(random_number).zippyshare)
# ================================================================ #

import os
from IPython.display import clear_output
from google.colab import files

if mode == 'single':
  if direct_url == '':
    print("The URL field is empty!")
  else:
    if store_path == '':
      store_path = '/content/downloads/zippyshare'
      !plowdown {direct_url} -o {store_path}
    else:
      !plowdown {direct_url} -o {store_path}
elif mode == 'batch':
  print("Upload a download.txt file that contains a list of zippyshare links.\n")
  files.upload()
  clear_output()
  if store_path == '':
    store_path = '/content/downloads/zippyshare'
    !plowdown {direct_url} -o {store_path}
  else:
    !plowdown {direct_url} -o {store_path}

## AnonFiles Uploader

In [ ]:
# ============================= FORM ============================= #
# @markdown <p><font size=5px>← [Start] Upload to anonfiles</font></p>
file_path = "" # @param {type: "string"}

url = "https://api.anonfiles.com/upload"
# ================================================================ #

import requests

x = requests.post(url, files = {'file': open(file_path,'rb')},)

print("Download link: " + x.json()["data"]["file"]["url"]["full"])

## Create / Extract Archive

In [ ]:
# ============================= FORM ============================= #
# @markdown <p><font size=5px>← Install the Tools</font></p>
# ================================================================ #

import os
import IPython
from IPython.display import clear_output

os.system("sudo apt update")
os.system("apt install p7zip-full p7zip-rar unrar rar")

clear_output()

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5px>← Create archive</font>
source_path = "" #@param {type:"string"}
archive_type = "zip" #@param ["zip", "7z", "rar", "tar", "tar.gz"]
archive_name = "" #@param {type:"string"}
#@markdown > If left empty, the default name will be used (archive)
archive_password = "" #@param {type:"string"}
#@markdown > Leave this field empty if you do not want to protect the archive with password.
compression_level = "no_compression" #@param ["no_compression", "fastest", "fast", "normal", "maximum", "ultra"]
output_path = "" #@param {type:"string"}
#@markdown > If left empty, the default path will be used (/content)

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

from IPython.display import HTML, clear_output
import os, sys, re


if archive_name == "":
    archive_name = "archive"
else:
    pass

if archive_password == "":
    pass
else:
    archive_password = "-p" + archive_password

if compression_level == "no_compression":
    compression_level = "-mx=0"
elif compression_level == "fastest":
    compression_level = "-mx=1"
elif compression_level == "fast":
    compression_level = "-mx=3"
elif compression_level == "normal":
    compression_level = "-mx=5"
elif compression_level == "maximum":
    compression_level = "-mx=7"
elif compression_level == "ultra":
    compression_level = "-mx=9"

if output_path == "":
    output_path = "/content"
else:
    pass


if archive_type == "zip":
    if source_path == "":
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The source_path field is empty!</h2><br></center>"))
    else:
        #output_file_path = re.search("^[\/].+\/", source_path)
        #output_file_path_raw = output_file_path.group(0)
        #delsplit = re.search("\/(?:.(?!\/))+$", source_path)
        #folder_name = re.sub("^[\/]", "", delsplit.group(0))

        #os.environ['inputDir'] = source_path
        #os.environ['outputPath'] = output_file_path_raw
        #os.environ['folderName'] = folder_name
        #os.environ['archiveLevel'] = compression_level
        #os.environ['archivePassword'] = archive_password

        #!7z a -tzip "$archiveLevel" "$archivePassword" "$outputPath"/"$folderName".zip "$inputDirectory"
        !7z a -tzip "$compression_level" "$archive_password" "$output_path"/"$archive_name".zip "$source_path"
else:
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ More archive format will be added in the future.</h2><br></center>"))


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5px>← Extract archive</font>
archive_path = "" #@param {type:"string"}
archive_type = "zip" #@param ["zip", "7z", "rar", "tar", "gzip", "iso"]
archive_password = "" #@param {type:"string"}
#@markdown > Leave the archive_password field empty if archive is not password protected.
output_path = "" #@param {type:"string"}
#@markdown > Leave the output_path field empty to use default extraction path (/content).

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

import os, sys, re
from IPython.display import HTML, clear_output


if archive_password == "":
    pass
elif not archive_password == "":
    archive_password = "-p" + archive_password

if output_path == "":
    output_path = "-o/content"
elif output_path == "/content":
    output_path = "-o/content"
else:
    output_path = "-o" + output_path


os.environ['inputFile'] = archive_path
os.environ['inputPassword'] = archive_password
os.environ['outputFile'] = output_path


if archive_path == "":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The archive_path field is empty!</h2><br></center>"))
else:
    if archive_type == "zip":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    elif archive_type == "iso":
        !7z x "$inputFile" "$outputFile"
    elif archive_type == "rar":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    if archive_type == "tar":
        !7z x "$inputFile" "$outputFile"
    if archive_type == "7z":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    else:
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ More archive format will be added in the future.</h2><br></center>"))


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass

# Gallery-DL

<font size="4">Not all features and/or functions are implemented yet!  
I cannot yet think of a way to handle sites that require OAuth (for example, Pixiv).  

You could, however, use gallery-dl on a separate cell manually.
<br> To install in gdrive change this : <br>
self.download_path = pathlib.Path('/content/drive/My Drive/')
<br> OneDrive: <br>
self.download_path = pathlib.Path('/content/drives/Torrent')
```
!gallery-dl {command goes here}
```

In [ ]:
# ------------------------------#------------------------------ #
# @markdown <font size="5">← [Install] gallery-dl</font>
# ------------------------------#------------------------------ #

import subprocess
import pathlib
import shutil
import sys
from IPython.display import clear_output
from google.colab import files


class gallerydl:
    def __init__(self):
        clear_output()

        self.download_path = pathlib.Path('/content/drives/Torrent')
        self.gallery_path = self.download_path.joinpath('gallery-dl')
        self.cookies_path = self.gallery_path.joinpath('cookies.txt')

        return None


    def install(self):
        try:
            subprocess.run('pip install --upgrade pip setuptools wheel', shell=True)
            subprocess.run('pip install --upgrade requests', shell=True)
            subprocess.run('pip install -U -I --no-deps --no-cache-dir https://github.com/mikf/gallery-dl/archive/master.tar.gz', shell=True)
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')
            return exit
        else:
            clear_output()

        try:
            self.gallery_path.mkdir(parents=True, exist_ok=True)
        except FileExistsError:
            return None
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')
            return exit
        else:
            return None


    def check_cookies(self):
        if self.cookies_path.exists() and self.cookies_path.is_file() is True:
            return True
        else:
            return False


    def upload_cookies(self):
        if self.check_cookies() is True:
            self.remove_cookies()
        else:
            pass

        uploaded = files.upload()

        for filename in uploaded.keys():
            shutil.move(filename, self.gallery_path)

            print(f'[Notice]: Moved file "{filename}" to "{self.gallery_path}"')

        return None


    def remove_cookies(self):
        if self.check_cookies() is True:
            try:
                self.cookies_path.unlink()
            except FileNotFoundError:
                pass
            except Exception as unknown_error:
                print(f'[Error]: {unknown_error}')
                return exit
            else:
                pass
        else:
            pass

        return None


    def execute(self, command):
        #
        # Thank you to jfs from stackoverflow for the solution
        # to print out subprocess output line-by-line on-the-fly.
        # https://stackoverflow.com/a/28319191/14304638
        #

        try:
            with subprocess.Popen(command, shell=True, cwd=self.download_path, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, universal_newlines=True) as process:
                for line in process.stdout:
                    print(line, end='')

            if process.returncode != 0:
                raise subprocess.CalledProcessError(process.returncode, process.args)
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')
            return exit
        else:
            print(
                '\n' + ('-' * 100) + '\n' + 
                f'[Notice]: Downloads are stored in "{self.gallery_path}"' + 
                '\n' + ('-' * 100)
            )
            return exit


    def stop_cell(self):
        clear_output()
        return exit


if __name__ == '__main__':
    try:
        gallerydl().install()
    except KeyboardInterrupt:
        stop_cell()
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        exit
    else:
        pass


In [ ]:
# ------------------------------#------------------------------ #
# @markdown <font size="5">← [Start] gallery-dl</font>
download_url = "--filesize-min 30M --retries 1 --http-timeout 10.0 " #@param {type:"string"}

# @markdown ---

# @markdown <font size="5"><center>Additional Options</center></font>
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
use_login = False #@param {type:"boolean"}
use_cookies = False #@param {type:"boolean"}
# ------------------------------#------------------------------ #

from IPython.display import clear_output


def main():
    if download_url == '':
        print('[Error]: download_url is empty!')
        return exit
    else:
        pass

    if use_login is True:
        if username == '':
            print('[Error]: username is empty!')
            return exit
        else:
            flag_username = f'--username {username}'

        if password == '':
            print('[Error]: password is empty!')
            return exit
        else:
            flag_password = f'--password {password}'
    else:
        flag_username = ''
        flag_password = ''
    
    if use_cookies is True:
        if gallerydl().check_cookies() is True:
            flag_cookies = f'--cookies "/content/drive/My Drive/gallery-dl/cookies.txt"'
        else:
            print('[Notice]: Execution aborted due to missing "cookies.txt" file.')
            return exit
    else:
        flag_cookies = ''

    try:
        command = f'gallery-dl {flag_username} {flag_password} {flag_cookies} {download_url}'
        gallerydl().execute(command)
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        return exit
    else:
        return None


if __name__  == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        clear_output()
        exit
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        exit
    else:
        pass


#### <font size="5">Settings -- Cookies</font>

<font size="3">(collapsible / expandable)</font>

In [ ]:
# ==============================#============================== #
# @markdown <font size="5">Upload cookies.txt</font>
# @markdown > Run this cell if you want to manually upload or re-upload your "cookies.txt" file.
# ==============================#============================== #

from IPython.display import clear_output


if __name__ == '__main__':
    try:
        gallerydl().upload_cookies()
    except KeyboardInterrupt:
        gallerydl().stop_cell()
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        exit
    else:
        clear_output()


In [ ]:
# ==============================#============================== #
# @markdown <font size="5">Delete cookies.txt</font>
# @markdown > Run this cell if you want to delete your "cookies.txt" file.
# ==============================#============================== #

if __name__ == '__main__':
    try:
        gallerydl().remove_cookies()
    except KeyboardInterrupt:
        gallerydl().stop_cell()
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        exit
    else:
        pass


# FFmpeg video tools

### ***Required to use Scripts:*** Install FFmpeg, VCSI & Mkvtoolnix

In [ ]:
#@markdown <h3>← Click Here to Install FFmpeg, VCSI, Mkvtoolnix, Firefox, Furiousmount & Handbrake</h3>

#@title <font size="5">← ឵឵<i>Upgrade FFmpeg to v4.2.2</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
#os.system('pip install git+git://github.com/AWConant/jikanpy.git') //GPU Not supported
#os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4') //GPU Not supported
os.system('apt-get update')
os.system('apt-get install ffmpeg')
os.system('apt-get install mkvtoolnix')
os.system('pip install vcsi')
#os.system('sudo apt-get install synaptic')
#os.system('sudo apt install firefox')
os.system('sudo add-apt-repository ppa:stebbins/handbrake-releases -y')
os.system('sudo apt update -y')
os.system('sudo apt install --install-recommends handbrake-gtk handbrake-cli')
#os.system('sudo apt-get install furiusisomount')

clear_output()
print("Install Finished")

### » Re-encode a Video to a Different Resolution (*H265*) - Need GPU - Nvidia Telsa P100 or T4 (Support Both Single & Batch Processing)

In [ ]:
#@title Check GPU
#@markdown Run this to connect to a Colab Instance, and see what GPU Google gave you.

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print(gpu[1])
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

In [ ]:
path = "" #@param {type:"string"}
save_txt = False #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    !curl -s https://pastebin.com/raw/TApKLQfM -o "$txt"
    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()
    
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()

In [ ]:
import os, sys, re
#@markdown <b>Encoder</b>
Encoder = "GPU" #@param ["GPU", "CPU"]
codec = "x265" #@param ["x264", "x265"]
#@markdown <b>Encoding all videos in folder</b>
video_folder_path = '' #@param {type:"string"}
#@markdown ---
#@markdown <b>Encoding selected videos</b>
video_file_path1 = '' #@param {type:"string"}
video_file_path2 = '' #@param {type:"string"}
video_file_path3 = '' #@param {type:"string"}
video_file_path4 = '' #@param {type:"string"}
video_file_path5 = '' #@param {type:"string"}

#@markdown <b>Output Path</b>
output_file_path = "" #@param {type:"string"}

#counting
if video_file_path1 != "":
  coa = 1
else:
  coa = 0

if video_file_path2 != "":
  cob = 1
else:
  cob = 0

if video_file_path3 != "":
  coc = 1
else:
  coc = 0

if video_file_path4 != "":
  cod = 1
else:
  cod = 0

if video_file_path5 != "":
  coe = 1
else:
  coe = 0

#@markdown ---
resolution = 'same as input' #@param ["2160p","1440p","1080p", "720p", "480p", "360p", "240p", "same as input"]
encode_setting = 'Advance' #@param ["Advance", "HEVC", "HEVC 10 Bit"]
file_type = 'mkv' #@param ["mkv", "mp4"]
rip_audio = False #@param {type:"boolean"}
rip_subtitle = False #@param {type:"boolean"}

if rip_audio == False:
  rip_audio_string =  "-acodec copy"
else:
  rip_audio_string =  "-an"

if rip_subtitle == False:
  rip_subtitle_string = "-scodec copy"
else:
  rip_subtitle_string = "-sn"


if resolution == '2160p':
   w = '3840'
elif resolution == '1440p':
   w = '2560'
elif resolution == '1080p':
   w = '1980'
elif resolution == '720p':
   w = '1280'
elif resolution == '480p':
   w = '854'
elif resolution == '360p':
   w = '640'
elif resolution == '240p':
   w = '426'
else:
   w = ''

if  (w == '3840' or w == '2560' or w == '1980' or w == '1280' or w == '854' or w == '640' or w == '426'):
  scale_string = "-vf  scale="+(w)+":-1:flags=lanczos" 
else:
  scale_string = ""

ext = ".mp4",".MP4",".MTS",".mts",".m2ts",".mkv",".avi",".MOV",".mov",".wmv",".WMV",".flv",".mpg",".webm",".WEBM"
# As file at filePath is deleted now, so we should check if file exists or not not before deleting them
filePath = "ffmpeg.txt"
if os.path.exists(filePath):
     os.remove(filePath)

if video_folder_path == "":
 #try:
  f = open("ffmpeg.txt", "+w")
  x = (video_file_path1) + "\n" + (video_file_path2) + "\n" +(video_file_path3) + "\n" +(video_file_path4) +"\n" + (video_file_path5)
  f.write(x)
  f.close()
  count = coa+cob+coc+cod+coe
 #except:
  #err = 1

else:
#writing temp file
 for file in os.listdir(video_folder_path):
    if file.endswith(tuple(ext)):
        
     x = os.path.join(video_folder_path, file) 
     #print(x)
     print(x, file=open("ffmpeg.txt", "+a"))    

#counting line
 thefilepath = "ffmpeg.txt"
 count = len(open(thefilepath).readlines(  ))

#@markdown ---
#@markdown <h4>Advance Settings</h4>
#@markdown <b>Video Setting</b>
preset = 'slow' #@param ["slow", "medium", "fast", "hq", "hp", "bd", "ll", "llhq", "llhp", "lossless", "losslesshp"]
level = '5.2' #@param ["default","4.1", "5.1", "5.2", "6.2"]
tier = 'main' #@param ["default","main", "high"]
#@markdown <code>Setting only for GPU Encoding</code>
profile = 'main' #@param ["main", "main10", "rext"]
pixfmt = 'yuv420p' #@param ["nv12", "yuv420p", "p010le", "yuv444p", "p016le", "yuv444p16le"]
rc = 'vbr_hq' #@param ["vbr", "cbr", "vbr_2pass", "ll_2pass_size", "vbr_hq", "cbr_hq"]
rcla = '32' #@param ["8", "16", "32", "64"]
overall_bitrate = 18000 #@param {type:"slider", min:2000, max:50000, step:100}
max_bitrate = 21000 #@param {type:"slider", min:2000, max:50000, step:100}
buffer_size = 60000 #@param {type:"slider", min:500, max:90000, step:100}
deblock = -3 #@param {type:"slider", min:-6, max:6, step:1}
reframe = 5 #@param {type:"slider", min:1, max:6, step:1}
surfaces = 64 #@param {type:"slider", min:0, max:64, step:1}
#@markdown <code>Setting only for CPU Encoding</code>
profile_cpu = 'main10' #@param ["main10"]
pixfmt_cpu = 'yuv420p10le' #@param ["yuv420p","yuv420p10le","yuv444p","yuv444p16le"]
threads = 16 #@param {type:"slider", min:0, max:16, step:1}
crf = 28 #@param {type:"slider", min:0, max:30, step:1}


if level != "default":
  l_string =  "-level "+str(level)
else:
  l_string =""

if tier != "default":
 t_string =  "-tier "+str(tier)
else:
 t_string = ""

#tp = '1' #@param ["0", "1"]
#cq = '21' #@param {type:"string"}
#qm ='21' #@param {type:"string"}
#qmx = '27' #@param {type:"string"}
#qp = '23' #@param {type:"string"}
#qb = '25' #@param {type:"string"}
#qi = '21' #@param {type:"string"}

#@markdown <b>Audio Setting</b>

audio_output = 'aac' #@param ["None", "copy", "flac", "aac", "libopus", "eac3", "No audio", "same as input"]
channel = 'same as input' #@param ["DownMix 2CH", "same as input"]

if audio_output == "same as input":
  audio_string = "-acodec copy"
elif audio_output == "No audio":
  audio_string = "-an"
elif audio_output == "None":
  audio_string = ""
else:
  audio_string = "-c:a "+(audio_output)

if channel == "DownMix 2CH":
  channel_string ="-ac 2"
else:
  channel_string =""

#@markdown <b>Subtitle Setting</b>
#@markdown <h5>Please use <code>ass</code> file for hardsub</h5>
hardsub = False #@param {type:"boolean"}
subtitle_option = 'same as input' #@param ["None","No sub", "Add custom sub","same as input"]
custom_subtitle_path = "" #@param {type:"string"}

#@markdown <b>Custom Added Setting</b>
custom_command = "-b:a 160k" #@param {type:"string"}


if hardsub == False:

 if subtitle_option == "No sub":
  subtitle_string = "-sn"
 elif subtitle_option == "same as input":
  subtitle_string = "-scodec copy"
 elif subtitle_option == "None":
  subtitle_string = ""
 else:
  subtitle_string = "-i "+(custom_subtitle_path)

else:
  subtitle_string = "ass="+(custom_subtitle_path)
#=================
if custom_command != "":
 c_string = custom_command
else:
 c_string = ""
#=================

os.environ['ps'] = preset
os.environ['pf'] = profile
os.environ['pf_cpu'] = profile_cpu
os.environ['pfm'] = pixfmt
os.environ['pfmcpu'] = pixfmt_cpu
os.environ['br'] = str(overall_bitrate)
os.environ['max'] = str(max_bitrate)
os.environ['buff'] = str(buffer_size)
os.environ['de'] = str(deblock)
os.environ['ref'] = str(reframe)
os.environ['sur'] = str(surfaces)
os.environ['lv'] = l_string
os.environ['ti'] = t_string
os.environ['rc'] = rc
os.environ['rl'] = rcla
os.environ['thr'] = str(threads)
os.environ['crf'] = str(crf)
os.environ['res'] = resolution
#os.environ['tp'] = tp
#os.environ['cq'] = cq
#os.environ['qP'] = qp
#os.environ['qB'] = qb
#os.environ['qI'] = qi
#os.environ['qm'] = qm
#os.environ['qmx'] = qmx
os.environ['scs'] = str(scale_string)
os.environ['aus'] = audio_string
os.environ['chc'] = channel_string
os.environ['sus'] = subtitle_string
os.environ['cus'] = str(c_string)
#=================
#Batch Encoding
if count != 0:
 f=open('ffmpeg.txt')
 lines=f.readlines()

 i = 0
 while i < count:
  video_file_path = lines[i]
  video_file_path = video_file_path.rstrip("\n")
  #print(video_file_path)

  delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
  testsplit = video_file_path.split("/")
  filename = re.sub("^[\/]", "", delsplit.group(0))
  filename_raw = re.sub(".{4}$", "", filename)
  resolution_raw = re.search("[^p]{3,4}", resolution)
  #output_file_path = re.search("^[\/].+\/", video_file_path)

  os.environ['inputFile'] = video_file_path
  os.environ['outputPath'] = output_file_path
  os.environ['fileName'] = filename_raw
  os.environ['fileType'] = file_type
  os.environ['resolutionWidth'] = resolution_raw.group(0)

  if Encoder == "GPU":
   if codec == "x265":
    if encode_setting == "Advance":

     !ffmpeg -hwaccel cuvid -i "$inputFile" -stats -flags +loop -c:v hevc_nvenc -preset:v "$ps" -rc "$rc" -2pass 1 -b:v "$br"k -maxrate "$max"k -bufsize "$buff"k -profile:v "$pf" $lv $ti -pix_fmt "$pfm" -rc-lookahead "$rl" -no-scenecut 1 -weighted_pred 1 -deblock:v "$de":"$de" -refs:v "$ref" -surfaces "$sur" $scs $aus $chs $sus $cus "$outputPath"/"$fileName"-"$res"."$fileType" 
  
    elif encode_setting == "HEVC":
     !ffmpeg -hwaccel cuvid -i "$inputFile" -stats -flags +loop -c:v hevc_nvenc -preset:v slow -rc vbr_hq -2pass 1 -b:v 2500k -maxrate 20M -bufsize 60M -cq 1 -forced-idr 1 -nonref_p 1 -pix_fmt p010le -rc-lookahead 32 -no-scenecut 1  -weighted_pred 1 -deblock:v -3:-3 -refs:v 5  -surfaces 64 $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

    else:
     !ffmpeg -hwaccel cuvid  -stats -flags +loop -c:v hevc_nvenc -preset:v slow -rc vbr_hq -2pass 1 -b:v 2500k -maxrate 20M -bufsize 60M -cq 1 -forced-idr 1 -nonref_p 1 -profile:v main10 -pix_fmt p010le -rc-lookahead 32 -no-scenecut 1  -weighted_pred 1 -deblock:v -3:-3 -refs:v 5  -surfaces 64 $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
   else:
     !ffmpeg -hwaccel cuvid -i "$inputFile" -c:v h264_cuvid $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
     
  else:
   if codec == "x265":
    if encode_setting == "Advance":
     !ffmpeg -i "$inputFile" -flags +loop -c:v libx265 -profile:v "$pf_cpu" $lv $ti -pix_fmt "$pfmcpu" -threads "$thr" -thread_type frame -preset:v "$ps" -crf "$crf" -x265-params "rc-lookahead=40:bframes=4:b-adapt=2:ref=6:aq-mode=0:aq-strength=0:aq-motion=0:me=hex:subme=3:max-merge=3:weightb=1:no-fast-intra=1:tskip-fast=0:rskip=0:strong-intra-smoothing=0:b-intra=1:early-skip=0:sao=0:rd=1:psy-rd=0:deblock=-5,-5" $scs $aus $chs  $sus $cus "$outputPath"/"$fileName"-"$res"."$fileType" 
   
    elif encode_setting == "HEVC":
     !ffmpeg -i "$inputFile" -c:v libx265 -crf 28 -threads 6 -thread_type frame $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

    else:
     !ffmpeg -i "$inputFile" -c:v libx265 -profile:v main10 -crf 28 -threads 6 -thread_type frame $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
   else:
     !ffmpeg -hide_banner -i "$inputFile" -c:v libx264 -preset "$ps" -crf "$crf" -threads "$thr" -strict experimental $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

  i += 1

 else:
  print("All Finished")
  os.remove(filePath)
else:
  print("Please input file or folder path")
#End of Code V1.5 - Codemater -  

# StreamLink

In [ ]:
#@markdown <center><h2>Streamlink Installer</h2></center><br>
#@markdown <center>Install StreamLink</center>
from IPython.display import clear_output
!pip install streamlink
clear_output()

In [ ]:
#@markdown <center><h3><h2>Stream Recorder</h2>Record any stream</h3></center><br>
#@markdown 1- Go to your Google Drive and create a directory called "stream" (whitout quotes).
#@markdown <br />2- Mount Gdrive above.
#@markdown <br />3- Install Streamlink above.
#@markdown <br />4- Start recording bellow.
import os
import time
import subprocess
from IPython.display import clear_output
RUN_Dir = "/content/tools/stream"
 
if not RUN_Dir:
  RUN_Dir = "tools/stream"
os.makedirs(RUN_Dir, exist_ok=True)

#code to use plugin that was removed from streamlink, remove the comment bellow and set the .py name
#if not os.path.exists(f"{RUN_Dir}/.py"):
#  !cd {RUN_Dir} && wget https://raw.githubusercontent.com/limontec/streamlink/master/src/streamlink/plugins/.py

#@markdown ---
#@markdown ####Type stream url
streamURL =  ""  # @param {type:"string"}
#@markdown ---
#@markdown ####Type any name for the stream
name = ""  # @param {type:"string"}
#@markdown ---
#@markdown ####Settings
#@markdown start stream record
start_record = True #@param {type:"boolean"}
#@markdown try restart record after stream disconnect
loop = False #@param {type:"boolean"}
#@markdown upload video file to your Drive after finish every record
upload_record = True #@param {type:"boolean"}
#@markdown upload all videos in this session to your Drive
upload_all = False #@param {type:"boolean"}
#@markdown ---
#@markdown #####Sites supported: https://streamlink.github.io/plugin_matrix.html
#@markdown ---

def record():
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  if start_record:
    !cd {RUN_Dir} && streamlink --plugin-dirs {RUN_Dir} {streamURL} best -o {RUN_Dir}/{name}{tempo}.mp4 --retry-streams 30 --retry-max 300
  upload(tempo)

def upload(tempo):
  if(os.path.exists('/content/drive')):
    if (upload_record and os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')) or loop:
      try:
        if(os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')):
          print("Copying "+name+tempo+".mp4 to drive...")
          !cd {RUN_Dir} && cp {name}{tempo}.mp4 /content/drive/My\ Drive/stream
        else:
          print("No file.")
      except Exception:
        print("Error!")
    elif upload_all:
      try:  
        print("Copying *.mp4 to drive...")
        os.system('cd '+RUN_Dir+' && cp *.mp4 /content/drive/My\ Drive/stream')
      except Exception:
        print("Error!")
    else:
      print("Nothing copied.")
  else:
    print("You need to mount Gdrive to /content/drive to upload")

record()
while loop:
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  record()
  clear_output()
